In [182]:
import pickle as pkl
import datetime

import csv
# Scraping items
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os

from IPython.core.display import display, HTML
import concurrent.futures


import numpy as np
from scipy import stats

import pandas as pd
from subprocess import call
import concurrent.futures
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#from pymongo import MongoClient

In [128]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [348]:
with open('SF_Urls','r') as f:
    urls=pd.read_csv(f)
urls2=[x.strip().strip('\'').strip('\'') for x in list(urls.columns)]

In [ ]:
museums='https://www.tripadvisor.com/Attractions-g60713-Activities-c49-San_Francisco_California.html#FILTERED_LIST'
museums2='https://www.tripadvisor.com/Attractions-g60713-Activities-c49-oa30-San_Francisco_California.html#FILTERED_LIST'
#list goes up to oa90
#
sites_and_attractions='https://www.tripadvisor.com/Attractions-g60713-Activities-c47-San_Francisco_California.html'
s_and_a='https://www.tripadvisor.com/Attractions-g60713-Activities-c47-oa30-San_Francisco_California.html#FILTERED_LIST'
#goes up to 

sf_attractions='https://www.tripadvisor.com/Attractions-g60713-Activities-San_Francisco_California.html'
sf_attractions2='https://www.tripadvisor.com/Attractions-g60713-Activities-oa30-San_Francisco_California.html#FILTERED_LIST'
#up to 'oa720'

In [273]:
driver = webdriver.Chrome(chromedriver)

In [276]:
sf_attraction_template='https://www.tripadvisor.com/Attractions-g60713-Activities-oa{}-San_Francisco_California.html#FILTERED_LIST'
tripsf_template='https://trip.sftravel.com/plan-travel/united-states/san-francisco/?page={}'
trip_sf_urls=[tripsf_template.format(i) for i in range(1,89)]
sf_attraction_urls=[sf_attraction_template.format(30*i) for i in range(1,10)]

In [277]:
sf_attraction_urls

['https://www.tripadvisor.com/Attractions-g60713-Activities-oa30-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa60-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa90-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa120-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa150-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa180-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa210-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa240-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa270-San_Francisco_California.html#FILTERED_LI

In [278]:
driver.get(sf_attraction_urls[0])

In [279]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [280]:
individual_urls=soup.find_all('div', class_='listing_title')

In [294]:
url_final=[]
for iu in individual_urls:
    partial_url=iu.find('a')['href']
    full_url='https://www.tripadvisor.com'+partial_url
    url_final.append(full_url)

In [306]:
initial_page=sf_attraction_template.format('')
other_pages=[sf_attraction_template.format(30*i) for i in range(1,10)]
all_pages=[initial_page]+other_pages
all_pages

['https://www.tripadvisor.com/Attractions-g60713-Activities-oa-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa30-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa60-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa90-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa120-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa150-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa180-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa210-San_Francisco_California.html#FILTERED_LIST',
 'https://www.tripadvisor.com/Attractions-g60713-Activities-oa240-San_Francisco_California.html#FILTERED_LIST'

In [311]:
def make_url_list(sf_attraction_template):
    
    #The inital and other pages are slightly different so we just break these up
    
    initial_page=sf_attraction_template.format('')
    other_pages=[sf_attraction_template.format(30*i) for i in range(1,25)]
    all_pages=[initial_page]+other_pages
    
    urls_final=[]
    
    for p in all_pages:
        
        try:
            driver.get(p)
            soup=BeautifulSoup(driver.page_source,'html.parser')
            individual_urls=soup.find_all('div', class_='listing_title')

            for iu in individual_urls:
                partial_url=iu.find('a')['href']
                full_url='https://www.tripadvisor.com'+partial_url
                urls_final.append(full_url)
        except:
            pass
        
    return urls_final
    

In [312]:
urls_final=make_url_list(sf_attraction_template)

In [314]:
with open('urls_final.pkl','wb') as f:
    pkl.dump(urls_final,f)

In [319]:
driver.get(urls_final[15])

In [320]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [337]:
rating=soup.find('span',class_='ui_bubble_rating bubble_40')['alt'][0]
tags=[x.text for x in soup.find('div',class_='detail').find_all('a',href=True)]
name=soup.find(id='HEADING').text

In [180]:
location_features={}
for url in trip_sf_urls:
    driver.get(url)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    for location in soup.find_all('div',class_='items-item'):
        try:
            features=location.find(class_='items-item-category hidden-xs').text
            name=location.find(class_='items-item-link').text
            location_features[name]=features
        except:
            pass
        

In [268]:
def make_feature_dct(ta_to_sf):
    ta_feature_dct={}
    ta_places=list(ta_to_sf.keys())
    for ta_place in ta_places:
        ta_feature_dct[ta_place]=location_features[ta_to_sf[ta_place]]
    return ta_feature_dct
    

In [270]:
with open('ta_feature_dct.pkl','wb') as f:
    pkl.dump(ta_feature_dct,f)

In [193]:
with open('location_features.pkl','wb') as f:
    pkl.dump(location_features,f)

In [131]:
driver.get(urls2[3])

In [7]:
soup=BeautifulSoup(driver.page_source,'html.parser')


In [6]:
file='data/SF_authors.pkl'
with open(file,'rb') as f:
    names=pkl.load(f)

In [351]:
len(names)

54729

In [352]:
urls=['https://www.tripadvisor.com/members/'+x for x in names]

In [16]:
driver.get(urls[5])

In [18]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [111]:
stuff=soup.find_all('li',class_='cs-review')
type(stuff[0])

bs4.element.Tag

In [8]:
attraction_name=stuff[1].find_all(class_='cs-review-location')[0].text
attraction_name

NameError: name 'stuff' is not defined

In [35]:
rating=stuff[0].find_all(class_='cs-review-rating')[0].find('span')
type(rating)

bs4.element.Tag

In [356]:
urls[2][36:]

'Jacob S'

In [101]:
def get_data(url):
    users={}
    
    name=url[36:]
    driver.get(url)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    reviews=soup.find_all('li',class_='cs-review')
    ranks={}
    for review in reviews:

        try:
            attraction_name=review.find_all(class_='cs-review-location')[0].text
            rating=int([str(tag) for tag in review.find_all()][-3][-10:-9])
        except:
            pass
        #rating=0
        if 'San Francisco:' == attraction_name[:14]:
            ranks[attraction_name]=rating
    users[name]=ranks
    return {name,ranks}
    

In [ ]:
sftrip_data=get_data()

In [89]:
counts = []
with concurrent.futures.ProcessPoolExecutor() as executor:
    well = [executor.submit(get_data, url) for url in urls[:10]]
    counts.append(well)

In [358]:
def scrape_reviews(urls):
    count=0
    tmp={}
    users={}
    for url in urls:
        name=url[36:]
        driver.get(url)
        soup=BeautifulSoup(driver.page_source,'html.parser')
        reviews=soup.find_all('li',class_='cs-review')
        ranks={}
        for review in reviews:

            try:
                attraction_name=review.find_all(class_='cs-review-location')[0].text
                rating=int([str(tag) for tag in review.find_all()][-3][-10:-9])
            except:
                pass
            #rating=0
            if 'San Francisco:' == attraction_name[:14]:
                ranks[attraction_name]=rating
        users[name]=ranks
        tmp[name]=ranks
        count+=1
        if count%100==0:
            with open(f'data/file_{9900+count}.pkl','wb') as f:
                pkl.dump(tmp,f)
            print(count)
            tmp={}
    
    return users
        
    
    

In [ ]:
users=scrape_reviews(urls[9900:])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [255]:
def make_matrix(users,ta_to_sf):
    user_list=list(users.keys())
    # Need full list of SF locations
    places=[]
    for user_key in users.keys():
        for place in users[user_key].keys():
            place=place[15:]
            if place in list(ta_to_sf.keys()):
                places.append(place)
    places=list(set(places))


    # Make array for user1
    matrix=[]
    for u in user_list:
        user_likes=[]
        for p in places:
            if p in users[u].keys():
                user_likes.append(users[u][p])
            else:
                user_likes.append(float('NaN'))
        matrix.append(user_likes)

    return np.array(matrix), user_list, places

In [199]:
results=[]
for k in range(100,10000,100):
    with open(f'data/file_{k}.pkl','rb') as f:
        likes=pkl.load(f)
        results.append(likes)
        
        
res=results[0]
for i in range(1,len(results)):
    res={**res,**results[i]}

In [256]:
matrix, user_list, places = make_matrix(res,ta_to_sf)

In [340]:
places

['54 Mint',
 'Tonga Room',
 'Albona Ristorante Istriano',
 'Redwood Ramblers',
 'Legion of Honor Cafe',
 'Ghirardelli Square',
 'Alcatraz Cruises, LLC',
 'Tin How Temple',
 'La Folie',
 'The Cartwright Hotel - Union Square, BW Premier Collection',
 "Cruisin' The Castro Walking Tours",
 'Madame Tussauds San Francisco',
 'Tower Tours - San Francisco Sightseeing Specialist',
 'Klyde Cafe & Wine Bar',
 'Ghirardelli Ice Cream & Chocolate Shop',
 "McCormick & Kuleto's Seafood & Steaks",
 'Catch',
 'Le Central',
 'Ragazza',
 'Beach Blanket Babylon',
 'Aquarium of the Bay',
 'Sutro Baths',
 'Sightglass Coffee',
 'Super Duper Burgers',
 'Buena Vista',
 'Rich Table',
 'San Francisco Love Tours',
 "Bloomingdale's",
 'The Baked Bear - Custom Ice Cream Sandwiches',
 'The Bell Tower',
 'Marlowe',
 'Izakaya Sozai',
 "Ben & Jerry's",
 'San Francisco Fire Engine Tours & Adventures',
 'Blue Bottle Cafe',
 "Morton's The Steakhouse",
 'Mikkeller Bar',
 "Alexander's Steakhouse",
 "St. Mary's Cathedral",
 '

In [209]:
places_clean=[x[15:] for x in places]

In [249]:
# want to reduce places_clean list
def make_ta_to_sf(places_clean,sftrip_locations):
    ta_to_sf={}
    for pc in places_clean:
        comp_best=0
        current_closest=None
        for sft in sftrip_locations:
            comp=fuzz.partial_ratio(pc.lower(),sft.lower())
            if comp>comp_best:
                comp_best=comp
                current_closest=sft

        if comp_best>90:
            ta_to_sf[pc]=current_closest
    return ta_to_sf
        

In [242]:
with open('ta_to_sf.pkl','wb') as f:
    pkl.dump(ta_to_sf,f)

In [215]:
sftrip_locations=list(location_features.keys())

In [37]:
with open('likes.pkl','wb') as f:
    pkl.dump(np.array(matrix),f)